Objective - Working with Text Data 


In [1]:
arr =[1,2,3,4]
print(arr)

[1, 2, 3, 4]


In [2]:
print("Hello")

Hello


In [1]:
import torch, tiktoken
print(torch.__version__)
print(tiktoken.__version__)


d:\LLMs-from-scratch\.venv\lib\site-packages\torch\_subclasses\functional_tensor.py:283: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


2.10.0+cpu
0.12.0


Package that are being used in this notebook.

In [4]:
from importlib.metadata import version
print("torch version: ", version("torch"))
print("tiktoken version: ", version("tiktoken"))

torch version:  2.10.0
tiktoken version:  0.12.0


This chapter covers data preparation and sampling to get input data "ready" from the LLM.

In [4]:
print("hi")

hi


In [5]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text= f.read()
print("Total number of character: ", len(raw_text))
print(raw_text[:99])

Total number of character:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


* The goal is to tokenize and embed this for an LLM
* Let's develop a simple tokenizer based on some simple sample text that we can then later apply to the above 
* The following regular expression will split on whitespaces



In [6]:
import re
text = "Hello IIT Patna , This is an AI era"
result = re.split(r'(\s)', text)
print(result)

['Hello', ' ', 'IIT', ' ', 'Patna', ' ', ',', ' ', 'This', ' ', 'is', ' ', 'an', ' ', 'AI', ' ', 'era']


* We don't only want to split on whitespace but also commas and periods, so let's modify the regular expression to do that as well


In [7]:
result = re.split(r'([,.\)])', text)
print(result)

['Hello IIT Patna ', ',', ' This is an AI era']


* As we can see, this creates empty string, let's remove them

In [8]:
# Strip whitespace from each item and then filter out any empty string.
result =[item for item in result if item.strip()]
print(result)

['Hello IIT Patna ', ',', ' This is an AI era']


In [9]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed= [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


* Let's calculate the total number of tokens

In [10]:
print(len(preprocessed))

4690


* Converting tokens into token IDs


In [11]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [12]:
vocab = {token: integer for integer, token in enumerate(all_words)}


* Below are the first 50 entries in this vocabulary

In [13]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [14]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
     self.str_to_int = vocab
     self.int_to_str= {i:s for s, i in vocab.items()}

    def encoded(self, text):
       preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

       preprocessed =[
          item.strip() for item in preprocessed if item.strip()
       ]
       ids = [self.str_to_int[s] for s in preprocessed]
       return ids
    
    def decode(self, ids):
       text = " ".join([self.int_to_str[i] for i in ids])
       text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
       return text


* The `encode` function turns text into token IDs
* The `decode` function turns token IDs back into text

In [15]:
tokenizer = SimpleTokenizerV1(vocab)
text = """It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encoded(text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


* We can decode the integers back into text

In [16]:
tokenizer.decode(ids)

'It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [17]:
tokenizer.decode(tokenizer.encoded(text))

'It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

* Let's see what happens if we tokenize the following text:

In [35]:
# tokenizer = SimpleTokenizerV1(vocab)
# text ="Hello, do you like tea. Is this-- a test?"
# tokenizer.encoded(text)

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab={token: integer for integer, token in enumerate(all_tokens)}


In [37]:
len(vocab.items())

1132

In [21]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


* We also need to adjust the tokenizer according so that it knows when and how to use the new `<unk>` token

In [22]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
     self.str_to_int = vocab
     self.int_to_str= {i:s for s, i in vocab.items()}

    def encoded(self, text):
       preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

       preprocessed =[
          item.strip() for item in preprocessed if item.strip()
       ]
       preprocessed=[
          item if item in self.str_to_int
          else "<|unk|>" for item in preprocessed
       ]
       ids = [self.str_to_int[s] for s in preprocessed]
       return ids
    
    def decode(self, ids):
       text = " ".join([self.int_to_str[i] for i in ids])
       text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
       return text


* Let's try to tokenize text with the modified tokenizer

In [23]:
tokenizer = SimpleTokenizerV2(vocab)
text1= "Hello, do you like tea?"
text2="In the sunlit terraces of the palaces."
text=" <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palaces.


In [24]:
tokenizer.encoded(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [25]:
tokenizer.decode(tokenizer.encoded(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

In [ ]:
#pip install tiktoken

In [26]:
import importlib
import tiktoken
print("tiktoken version: ", importlib.metadata.version("tiktoken"))


tiktoken version:  0.12.0


In [27]:
tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
text=(
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [30]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


* Data sampling with a sliding window

* We `train` LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict:

In [32]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text=f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


* For each text chunk, we want the input and targets
* Since we want the model to predict the next word, the targets are the inputs shifted by one position to the right

In [33]:
enc_sample = enc_text[50:]

In [34]:
context_size = 4
x = enc_sample[:context_size]
y= enc_sample[:context_size+1]
print(f"x: {x}")
print(f"y:   {y}")



x: [290, 4920, 2241, 287]
y:   [290, 4920, 2241, 287, 257]


* one by one, the prediction would look like as follows

In [35]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired=enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [36]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired=enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


* Install and import PyTorch

In [37]:
import torch
print("PyTorch version: ", torch.__version__)

d:\LLMs-from-scratch\.venv\lib\site-packages\torch\_subclasses\functional_tensor.py:283: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


PyTorch version:  2.10.0+cpu


* We use a sliding window approch, changing the position by + 1

* Create dataset and dataloader that extract chunks from the input text dataset

In [38]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [39]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

* Let's test the dataloader with a batch size of 1 for an LLM with a context size of 4:

In [40]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text=f.read()

In [41]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [42]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


* We can also create batched outputs
* Note that we increase the stride here so that we don't have overlaps between the batches, since more overlap could lead to increased overfitting

In [44]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


* Creating token embedding

In [45]:
inputs_ids = torch.tensor([2,3,5,1])

* For the sake of simplicity, suppose we have a small vocabulary of only 6 words and we want create emabedding of size 3:

In [46]:
vocab_size = 6
output_dim= 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

* This would result in a 6 x 3 weight matrix.

In [47]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


* To convert a token with id 3 into a 3-dimensional vector, we do the following

In [48]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


* To embed all four `input_ids` value above , we do

In [49]:
print(embedding_layer(inputs_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


* Encoding word positions

In [50]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

* If we sample data from the dataloader, we embed the tokens in each batch into a 256-dimensional vector
* If we have a batch size of 8 with 4 tokens each, this results in a 8 x 4 x 256 tensor:

In [51]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [52]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [53]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(token_embeddings)

torch.Size([8, 4, 256])


* GPT-2 uses absolute position embeddings, so we just create another embedding layer:

In [55]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# uncomment & execute the following line to see how the embedding layer weights look like
# print(pos_embedding_layer.weight)

Parameter containing:
tensor([[-0.8194,  0.5543, -0.8290,  ...,  0.1325,  0.2115,  0.3610],
        [ 0.4193, -0.9461, -0.3407,  ...,  0.7930,  1.7009,  0.5663],
        [-0.2362, -1.7187, -1.0489,  ...,  1.1218,  0.2796,  0.9912],
        [-0.9549,  0.4699,  0.2580,  ..., -1.3689,  1.6505,  1.3488]],
       requires_grad=True)


In [56]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(pos_embeddings)

torch.Size([4, 256])
tensor([[-0.8194,  0.5543, -0.8290,  ...,  0.1325,  0.2115,  0.3610],
        [ 0.4193, -0.9461, -0.3407,  ...,  0.7930,  1.7009,  0.5663],
        [-0.2362, -1.7187, -1.0489,  ...,  1.1218,  0.2796,  0.9912],
        [-0.9549,  0.4699,  0.2580,  ..., -1.3689,  1.6505,  1.3488]],
       grad_fn=<EmbeddingBackward0>)


* To create the input embeddings used in an LLM, we simply add the token and the positional embeddings:

In [57]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(input_embeddings)

torch.Size([8, 4, 256])


##----------------- Chap one end --------- ## Happy LLM Journey 